## Imports

In [16]:
import requests
import os

from urllib.request import urlopen
from zipfile import ZipFile

from os import listdir
from os.path import isfile, join

import pandas as pd
import glob

import altair as alt
alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

## Download and Unzip Data

In [2]:
def unzip(url, out_folder):
    try:
        tempzip = open("/tmp/tempfile.zip", "wb")
        tempzip.write(urlopen(url).read())
        tempzip.close()
        zf = ZipFile("/tmp/tempfile.zip")
        zf.extractall(path=out_folder)
        zf.close()
        print(f"Unzipped data to: {os.getcwd()}/{out_folder}")
    except Exception as req:
        raise UnzipFileError(req)

def get_data(url, out_folder):
    try:
        request = requests.get(url)
        if request.status_code == 200:
            if not os.path.exists(os.getcwd() + "/" + out_folder):
                os.makedirs(out_folder + "/")
        
    except Exception as req:
        print("Invalid URL provided.")
        print(req)

    return unzip(url, out_folder)

In [7]:
if not os.path.exists(os.getcwd() + "/data/raw/__MACOSX"):
    pass

else:
    files = glob.glob(os.getcwd() + "/data/raw/__MACOSX/*")
    for f in files:
        os.remove(f)
    os.rmdir(os.getcwd() + "/data/raw/__MACOSX")

if len(os.listdir(os.getcwd() + "/data/raw")) > 0:
    files = glob.glob(os.getcwd() + "/data/raw/*")
    for f in files:
        os.remove(f)
else:
    pass

In [8]:
time_get_data = %timeit -q -o -r 1 get_data(url="https://figshare.com/ndownloader/files/26766812", out_folder="data/raw")
print(f"Time Elapsed (to download and unzip the data): {time_get_data}")

Unzipped data to: /Users/jordancasoli/Documents/MDS/525/525-Group30/data/raw
Unzipped data to: /Users/jordancasoli/Documents/MDS/525/525-Group30/data/raw
Time Elapsed (to download and unzip the data): 2min 46s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Combining all .csv files into one DataFrame

In [9]:
path, dirs, files = next(os.walk("data/raw"))
file_count = len(files)

def combine_data():
    combined_df = pd.DataFrame()
    for file in files:
        temp = pd.DataFrame()
        model_name = file.split('_')[0]
        temp = pd.read_csv('data/raw/' + file)
        temp['model'] = model_name
        combined_df = pd.concat([combined_df, temp], axis=0)
    return combined_df

time_combine_data = %timeit -q -o -r 1 combine_data()
print(f"Time Elapsed (to combine the data): {time_combine_data}\n")

combined_df = combine_data()

combined_df.info()

Time Elapsed (to combine the data): 2min 19s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62513863 entries, 0 to 3541152
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   lat_min        float64
 2   lat_max        float64
 3   lon_min        float64
 4   lon_max        float64
 5   rain (mm/day)  float64
 6   model          object 
dtypes: float64(5), object(2)
memory usage: 3.7+ GB


In [10]:
if len(os.listdir(os.getcwd() + "/data/processed")) > 0:
    files = glob.glob(os.getcwd() + "/data/processed/*")
    for f in files:
        os.remove(f)

In [12]:
%%time
combined_df.to_csv('data/processed/combined_df.csv')

CPU times: user 5min 53s, sys: 7.11 s, total: 6min
Wall time: 6min 8s


In [8]:
combined_df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


# Load the combined CSV to memory and perform a simple EDA

In [5]:
%%time
df = pd.read_csv("data/processed/combined_df.csv", index_col=0)

MPI-ESM1-2-HR       5154240
NorESM2-MM          3541230
CMCC-CM2-SR5        3541230
TaiESM1             3541230
CMCC-ESM2           3541230
CMCC-CM2-HR4        3541230
SAM0-UNICON         3541153
GFDL-CM4            3219300
FGOALS-f3-L         3219300
GFDL-ESM4           3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
observed              46020
Name: model, dtype: int64
CPU times: user 54 s, sys: 17.6 s, total: 1min 11s
Wall time: 1min 34s


In [6]:
df.describe()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.929456e+07
mean,-3.310482e+01,-3.197757e+01,1.469059e+02,1.482150e+02,1.901827e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.588275e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.876672e-06
50%,-3.300000e+01,-3.204188e+01,1.468750e+02,1.481250e+02,6.161705e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.021314e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62513863 entries, 0 to 3541152
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   lat_min        float64
 2   lat_max        float64
 3   lon_min        float64
 4   lon_max        float64
 5   rain (mm/day)  float64
 6   model          object 
dtypes: float64(5), object(2)
memory usage: 3.7+ GB


Loading the whole dataset requires 3.7GB of memory! 

#### Loading only the columns we need

In [9]:
%%time
useful_cols = ['time','rain (mm/day)']
df_columns = pd.read_csv("data/processed/combined_df.csv",usecols=useful_cols)

CPU times: user 39.1 s, sys: 8.2 s, total: 47.3 s
Wall time: 55.7 s


In [10]:
df_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62513863 entries, 0 to 62513862
Data columns (total 2 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   rain (mm/day)  float64
dtypes: float64(1), object(1)
memory usage: 953.9+ MB


By only loading the two columns I'm interested in we were able to reduce memory usage from 3.7GB to 950MB. 

#### Changing the dtype 

It looks like we may be able to reduce the memory requirements by changing dtypes from float64 to float32. Let's see what happens...

In [11]:
print(f"Memory usage with float64: {df[['lat_min','lat_max','lon_min','lon_max','rain (mm/day)']].memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df[['lat_min','lat_max','lon_min','lon_max','rain (mm/day)']].astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 3000.67 MB
Memory usage with float32: 1750.39 MB


In [12]:
df_dtype = df.astype({'lat_min': 'float32',
                      'lat_max': 'float32',
                      'lon_min': 'float32',
                      'lon_max': 'float32',
                      'rain (mm/day)': 'float32'})

In [13]:
df_dtype.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62513863 entries, 0 to 3541152
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   lat_min        float32
 2   lat_max        float32
 3   lon_min        float32
 4   lon_max        float32
 5   rain (mm/day)  float32
 6   model          object 
dtypes: float32(5), object(2)
memory usage: 2.6+ GB


So by changing the dtype of a few columns we were in fact able to reduce the memory requirements from 3.7GB to around 2.6GB. 

#### Simple EDA

In [23]:
df_dtype['time']= pd.to_datetime(df_dtype['time'])
rain_by_month_df = df_dtype.groupby(pd.Grouper(key='time', axis=0, 
                      freq='M')).mean().head(12)
rain_by_month_df

,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
time,,,,,
1889-01-31,-33.104778,-31.97756,146.905869,148.214966,2.379870
1889-02-28,-33.104778,-31.97756,146.905869,148.214966,2.552886
1889-03-31,-33.104778,-31.97756,146.905869,148.214966,1.703958
1889-04-30,-33.104778,-31.97756,146.905869,148.214966,1.577898
1889-05-31,-33.104778,-31.97756,146.905869,148.214966,1.970277
1889-06-30,-33.104778,-31.97756,146.905869,148.214966,1.774999
1889-07-31,-33.104778,-31.97756,146.905869,148.214966,1.769375
1889-08-31,-33.104778,-31.97756,146.905869,148.214966,1.764908
1889-09-30,-33.104778,-31.97756,146.905869,148.214966,1.879251
